In [ ]:
# Import libraries

import sys
import pypsa
import logging
import pandas as pd
import numpy as np
import xarray as xr
from plotly import graph_objects as go
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [ ]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = False
h2_initial = 1000
biogas_limit = 0
load_target = 15

In [ ]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [ ]:
network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [ ]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

In [ ]:
renewable_generators = ['solar', 'onwind']
links_charge = ['battery-charge']
links_discharge = ['battery-discharge']

if use_offwind:
    renewable_generators += ['offwind']
if use_h2:
    links_charge += ['h2-electrolysis']
if use_h2 or biogas_limit > 0:
    links_discharge += ['gas-turbine']


# Calculate renewables distribution and cost distribution (helper for the LCOE further down)
energy = pd.DataFrame(columns=['energy_to_load', 'cost_to_load', 'energy_to_battery', 'cost_to_battery', 'energy_to_h2', 'cost_to_h2', 'total_energy', 'total_cost'])

energy['total_energy'] = network.generators_t.p[renewable_generators].sum() * 3
energy['total_cost'] = network.generators.loc[renewable_generators]['p_nom_opt']*network.generators.loc[renewable_generators]['capital_cost'] + energy['total_energy'] * network.generators.loc[renewable_generators]['marginal_cost']

energy['energy_to_load'] = (network.generators_t.p[renewable_generators] - network.generators_t.p[renewable_generators].div(
network.generators_t.p[renewable_generators].sum(axis=1), axis=0).mul(
    network.links_t.p0[links_charge].sum(axis=1), axis=0)).sum() * 3
energy['cost_to_load'] = energy['energy_to_load'] / energy['total_energy'] * energy['total_cost']

energy['energy_to_battery'] = network.generators_t.p[renewable_generators].div(network.generators_t.p[renewable_generators].sum(axis=1), axis=0).mul(network.links_t.p0['battery-charge'], axis=0).sum() * 3
energy['cost_to_battery'] = energy['energy_to_battery'] / energy['total_energy'] * energy['total_cost']

if use_h2:
    energy['energy_to_h2'] = network.generators_t.p[renewable_generators].div(network.generators_t.p[renewable_generators].sum(axis=1), axis=0).mul(network.links_t.p0['h2-electrolysis'], axis=0).sum() * 3
    energy['cost_to_h2'] = energy['energy_to_h2'] / energy['total_energy'] * energy['total_cost']

energy

In [ ]:
# Define the LCOE data frame
lcoe = pd.DataFrame(columns=['total_energy', 'total_cost', 'lcoe', 'curtailment'], index=['solar', 'onwind', 'offwind', 'battery', 'biogas', 'h2'])

# Add renewables to load (calculated above)
lcoe['total_energy'] = energy['energy_to_load']
lcoe['total_cost'] = energy['cost_to_load']

In [ ]:
# Battery calculations

# Add total energy output
lcoe.loc['battery', 'total_energy'] = -network.links_t.p1['battery-discharge'].sum() * 3

# Add electricity input cost
lcoe.loc['battery', 'total_cost'] = energy['cost_to_battery'].sum()
# Add inverter (modelled as links) capital costs (for now these have no marginal costs)
lcoe.loc['battery', 'total_cost'] += (network.links.loc[['battery-charge', 'battery-discharge']]['capital_cost']*network.links.loc[['battery-charge', 'battery-discharge']]['p_nom_opt']).sum()
# Add storage capital costs
lcoe.loc['battery', 'total_cost'] += network.stores.loc['battery', 'capital_cost'] * network.stores.loc['battery', 'e_nom_opt']
# Add storage marginal costs (do not include for now)
#stored_energy.loc['battery', 'total_cost'] += network.stores.loc['battery', 'marginal_cost'] * (-network.links_t.p1['battery-charge'].sum() * 3 * network.stores.loc['battery', 'marginal_cost'])


In [ ]:
#H2 calculations

if use_h2:
    if biogas_limit > 0:
        h2_gas_fraction = network.links_t.p0['H2 pipeline'].sum() / (network.generators_t.p[['biogas-market']].sum().values[0] + network.links_t.p0['H2 pipeline'].sum())
    else:
        h2_gas_fraction = 1

    # Add total energy output
    lcoe.loc['h2', 'total_energy'] = -network.links_t.p1['gas-turbine'].sum() * 3 * h2_gas_fraction

    # Add electricity input cost
    lcoe.loc['h2', 'total_cost'] = energy['cost_to_h2'].sum()
    # Add electrolysis (modelled as link) capital cost (for now this has no marginal costs)
    lcoe.loc['h2', 'total_cost'] += network.links.loc['h2-electrolysis', 'capital_cost'] * network.links.loc['h2-electrolysis','p_nom_opt']
    # Add storage capical cost (for not there is not marginal cost)
    lcoe.loc['h2', 'total_cost'] += network.stores.loc['h2', 'capital_cost'] * network.stores.loc['h2','e_nom_opt']
    # Add gas turbine (modelled as link) fractional capital cost (fraction of h2 in total gas)
    lcoe.loc['h2', 'total_cost'] += network.links.loc['gas-turbine', 'capital_cost'] * network.links.loc['gas-turbine','p_nom_opt'] * h2_gas_fraction
    # Add gas turbine (modelled as link) marginal cost
    lcoe.loc['h2', 'total_cost'] += network.links.loc['gas-turbine', 'marginal_cost'] * lcoe.loc['h2', 'total_energy']

In [ ]:
# Biogas calculations

if biogas_limit > 0:
    # Add total energy output
    lcoe.loc['biogas', 'total_energy'] = -network.links_t.p1['gas-turbine'].sum() * 3 * (1 - h2_gas_fraction)

    # Add biogas input cost
    lcoe.loc['biogas', 'total_cost'] = network.generators_t.p[['biogas-market']].sum().iloc[0] * network.generators.loc['biogas-market', 'marginal_cost'] * resolution
    # Add gas turbine (modelled as link) fractional capital cost (fraction of h2 in total gas)
    lcoe.loc['biogas', 'total_cost'] += network.links.loc['gas-turbine', 'capital_cost'] * network.links.loc['gas-turbine','p_nom_opt'] * (1 - h2_gas_fraction)
    # Add gas turbine (modelled as link) marginal cost
    lcoe.loc['biogas', 'total_cost'] += network.links.loc['gas-turbine', 'marginal_cost'] * lcoe.loc['biogas', 'total_energy']


In [ ]:
# Calculate LCOE per energy type

lcoe.round(9)

lcoe['lcoe'] = (lcoe['total_cost']/lcoe['total_energy']) / 1_000

# Show table
lcoe

In [ ]:
# Calculate and show overall LCOE for reference
lcoe['total_cost'].sum()/lcoe['total_energy'].sum() / 1000


### Validation

Below follows some code for validation of the LCOE calculation. The end point is an overall LCOE (not by energy type) that can be compared with the one above

In [ ]:
# Calculate generator costs
generator_capital_costs = (network.generators['p_nom_opt']*network.generators['capital_cost']).drop('backstop')
generator_marginal_costs = (network.generators_t.p.sum()*network.generators['marginal_cost']).drop('backstop') * resolution
generator_costs = generator_capital_costs + generator_marginal_costs

In [ ]:
# Calculate converter costs
converter_links = ['battery-charge', 'battery-discharge']
if use_h2 or biogas_limit > 0:
    converter_links += ['gas-turbine']
if use_h2:
    converter_links += ['h2_electrolysis']

converter_capital_costs = network.links.loc[converter_links]['p_nom_opt']*network.links.loc[converter_links]['capital_cost']
converter_marginal_costs = network.links_t.p0[converter_links].sum()*network.links.loc[converter_links]['marginal_cost']*resolution
if use_h2 or biogas_limit > 0:
    converter_marginal_costs.loc['gas-turbine'] = -network.links_t.p1['gas-turbine'].sum()*network.links.loc['gas-turbine', 'marginal_cost'] * resolution

converter_costs = converter_capital_costs + converter_marginal_costs

In [ ]:
# Calculate store costs (exclude marginal costs for now)
store_capital_costs = network.stores['e_nom_opt']*network.stores['capital_cost']
#store_marginal_costs = -network.links_t.p1['battery-charge'].sum()*3*network.stores.loc[['battery']]['marginal_cost']
#store_marginal_costs['h2'] = 0
store_costs = store_capital_costs #+ store_marginal_costs

In [ ]:
# Add up total costs, calculate total energy, and calculate the LCOE
total_costs = generator_costs.sum() + converter_costs.sum() + store_costs.sum()
total_energy = network.loads_t.p.sum()*3 - network.generators_t.p['backstop'].sum()*3
validation_lcoe = total_costs / total_energy['load'] / 1000

validation_lcoe

In [ ]:
# Comparison (should be zero)

round(lcoe['total_cost'].sum()/lcoe['total_energy'].sum() / 1000 - validation_lcoe, 10)